# Segment audios

In [1]:
import os

# Data Management
import pandas as pd

import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Audio
from pydub import AudioSegment

In [2]:
!ls ../../../../../mnt/e/Deep\ Learning\ Analysis/Predictions/

Fondeo2_Model31		      NOWOPAMDAY_Downsampled_Model23
NOWOPAMDAY_310102_06_Model31  WOPAMDAY_Model22


In [4]:
PREDICTIONS = "Fondeo2_Model31"

In [5]:
import pandas as pd

# Load CSV predictions.csv
df = pd.read_csv(f'../../../../../mnt/e/Deep Learning Analysis//Predictions/{PREDICTIONS}/predictions_no_background.csv')
df

,Path,Filename,StartSecond,EndSecond,MainClassification,ConfidenceScore,ConfidenceVector
0,Downsampled AQUI TAMBIÉN/Tarifa 2 SYLENCE/BUEN...,channelA_2024-05-27_21-43-19_48kHz.wav,288,291,Whistle,0.564021,"[0.435978502035141, 0.5640214681625366]"
1,Downsampled AQUI TAMBIÉN/Tarifa 2 SYLENCE/BUEN...,channelA_2024-05-27_21-48-19_48kHz.wav,0,3,Whistle,0.978356,"[0.02164352871477604, 0.9783564209938049]"
2,Downsampled AQUI TAMBIÉN/Tarifa 2 SYLENCE/BUEN...,channelA_2024-05-27_21-48-19_48kHz.wav,24,27,Whistle,0.559729,"[0.4402709901332855, 0.5597289800643921]"
3,Downsampled AQUI TAMBIÉN/Tarifa 2 SYLENCE/BUEN...,channelA_2024-05-27_21-48-19_48kHz.wav,42,45,Whistle,0.999319,"[0.0006812208448536694, 0.9993188381195068]"
4,Downsampled AQUI TAMBIÉN/Tarifa 2 SYLENCE/BUEN...,channelA_2024-05-27_21-48-19_48kHz.wav,48,51,Whistle,0.900744,"[0.09925549477338791, 0.9007444381713867]"
...,...,...,...,...,...,...,...
150,Downsampled AQUI TAMBIÉN/Tarifa 2 SYLENCE/BUEN...,channelA_2024-06-12_16-00-33_48kHz.wav,108,111,Whistle,0.619309,"[0.3806910216808319, 0.6193089485168457]"
151,Downsampled AQUI TAMBIÉN/Tarifa 2 SYLENCE/BUEN...,channelA_2024-06-12_16-15-33_48kHz.wav,27,30,Whistle,0.578439,"[0.42156097292900085, 0.5784390568733215]"
152,Downsampled AQUI TAMBIÉN/Tarifa 2 SYLENCE/BUEN...,channelA_2024-06-12_16-15-33_48kHz.wav,36,39,Whistle,0.614954,"[0.38504597544670105, 0.6149540543556213]"
153,Downsampled AQUI TAMBIÉN/Tarifa 2 SYLENCE/BUEN...,channelA_2024-06-12_16-15-33_48kHz.wav,42,45,Whistle,0.606559,"[0.393441379070282, 0.606558620929718]"


In [6]:
species = df["MainClassification"].unique()
species

array(['Whistle'], dtype=object)

In [86]:
# HARD_DRIVE = "f"
# OS = "WSL"

# import subprocess
# from NO_SHARABLE import PASSWORD

# if OS == "WSL":
#     command = f'mount -t drvfs F: /mnt/{HARD_DRIVE}'

#     try:
#         # Ejecutar el comando como sudo
#         proc = subprocess.Popen(
#             ['sudo', '-S'] + command.split(),
#             stdin=subprocess.PIPE,
#             stdout=subprocess.PIPE,
#             stderr=subprocess.PIPE,
#             universal_newlines=True
#         )

#         # Enviar la contraseña seguido de un salto de línea
#         stdout, stderr = proc.communicate(PASSWORD + '\n')

#         print('STDOUT:', stdout)
#         print('STDERR:', stderr)

#         if proc.returncode == 0:
#             print("Hard drive mounted successfully")
#             HARD_DRIVE = HARD_DRIVE
#         else:
#             print("Failed to mount hard drive")

#     except Exception as e:
#         print(f"An error occurred: {e}")

In [7]:
import os

In [8]:
for specie in species:
    # Load the subset
    subset = df[df["MainClassification"] == specie]
    subset = subset.reset_index(drop=True)
    print(len(subset))

155


In [9]:
import os
from pydub import AudioSegment
# import pandas as pd

In [10]:
# Load CSV predictions.csv
# df = pd.read_csv(f'../../../../../mnt/e/Deep Learning Analysis/Predictions/{PREDICTIONS}/predictions_to_validate.csv')

species = df["MainClassification"].unique()

# take for each MainClassification the 100 samples with the highest confidence
# df = df.sort_values(by=["ConfidenceScore"], ascending=False).groupby("MainClassification").head(100)

# count of rows per MainClassification
df["MainClassification"].value_counts()

MainClassification
Whistle    155
Name: count, dtype: int64

In [11]:
CLASSES_MAPPING = {
    'Background': 0,
    'Whistle': 1,
}

# Reverse mapping for easy lookup
IDX_TO_CLASSES = {v: k for k, v in CLASSES_MAPPING.items()}

In [94]:
# df["BackgroundConfidenceScore"] = df["ConfidenceVector"].apply(lambda x: eval(x)[0])

In [95]:
# df["ConfidenceSpecieBackground"] = df["ConfidenceScore"] - df["BackgroundConfidenceScore"]

In [13]:
def cut_audio(prediction, output_folder, audios_folder="../Audios/"):
    file = prediction["Path"]

    start_time = max(0, int(prediction["StartSecond"] * 1000) - 2000)  # Convert to ms
    end_time = min(1200000, int(prediction["EndSecond"] * 1000) + 2000)

    start_time = int(start_time)
    end_time = int(end_time)

    scientific_name = prediction["MainClassification"]
    confidence = prediction["ConfidenceScore"]
    # round to 2 decimals
    confidence = round(confidence, 2)

    # Get the output folder
    output_folder_specie = os.path.join(output_folder, scientific_name)
    os.makedirs(output_folder_specie, exist_ok=True)

    # Get the output file path
    base_filename = os.path.basename(file)
    output_file = os.path.join(
        output_folder_specie, f"{base_filename.replace('.wav', '')}_{start_time // 1000}_{end_time // 1000}_{confidence}.WAV"
    )

    if not os.path.exists(output_file):
        # Get the audio file
        audio_file = os.path.join(audios_folder, file)

        if not os.path.exists(audio_file):
            print(f"Warning: {audio_file} not found!")
            return

        # Load the audio file
        audio = AudioSegment.from_wav(audio_file)

        # Cut the audio segment
        cut_audio = audio[start_time:end_time]

        print(f"Exporting {output_file}...")

        # Export the cut audio segment
        cut_audio.export(output_file, format="wav")
    else:
        print(f"File {output_file} already exists")


AudioFolder = f"/mnt/e/Deep Learning Analysis/Segments/{PREDICTIONS}"

for specie in species:
    # Load the subset
    subset = df[df["MainClassification"] == specie]
    # reset index
    subset = subset.reset_index(drop=True)
    # For each subset saved in a csv, save in a different folder
    output_folder = f"{AudioFolder}/"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    # Cut the audio for each row in the subset
    for index, row in subset.iterrows():
        cut_audio(row, output_folder, audios_folder = f"/mnt/e/")

Exporting /mnt/e/Deep Learning Analysis/Segments/Fondeo2_Model31//Whistle/channelA_2024-05-27_21-43-19_48kHz_286_293_0.56.WAV...
Exporting /mnt/e/Deep Learning Analysis/Segments/Fondeo2_Model31//Whistle/channelA_2024-05-27_21-48-19_48kHz_0_5_0.98.WAV...
Exporting /mnt/e/Deep Learning Analysis/Segments/Fondeo2_Model31//Whistle/channelA_2024-05-27_21-48-19_48kHz_22_29_0.56.WAV...
Exporting /mnt/e/Deep Learning Analysis/Segments/Fondeo2_Model31//Whistle/channelA_2024-05-27_21-48-19_48kHz_40_47_1.0.WAV...
Exporting /mnt/e/Deep Learning Analysis/Segments/Fondeo2_Model31//Whistle/channelA_2024-05-27_21-48-19_48kHz_46_53_0.9.WAV...
Exporting /mnt/e/Deep Learning Analysis/Segments/Fondeo2_Model31//Whistle/channelA_2024-05-27_21-48-19_48kHz_49_56_0.81.WAV...
Exporting /mnt/e/Deep Learning Analysis/Segments/Fondeo2_Model31//Whistle/channelA_2024-05-27_21-48-19_48kHz_70_77_1.0.WAV...
Exporting /mnt/e/Deep Learning Analysis/Segments/Fondeo2_Model31//Whistle/channelA_2024-05-27_21-48-19_48kHz_79_86